# Class Definitions

The Turing Machine and Tape classes were originally sourced from https://python-course.eu/applications-python/turing-machine.php
although we added support for definitional output and file parsing of transitions among other capabbilities

In [ ]:
#
# set constant where to piuck up input files
#
FILE_DIR = '/Users/peter/My Drive/FAU/Computational Theory (COT4420)/turing_machine/config'
MAX_OPS = 250
DEBUG = True

In [ ]:

class Tape(object):
    
    #blank_symbol = " "
    
    def __init__(self,
                 tape_string = "",
                 blank_symbol = " ", debug=False):
        
        self.__tape = dict((enumerate(tape_string)))
        
        # last line is equivalent to the following three lines:
        #self.__tape = {}
        #for i in range(len(tape_string)):
        #    self.__tape[i] = input[i]
         
        self.__blank_symbol = blank_symbol
        
        if (debug):
            print('Tape::tape[' + str(self.__tape) + ']')
            print('Tape::blank_symbol[' + str(self.__blank_symbol) + ']')
        
        
    def get_blank_symbol(self):
        return self.__blank_symbol
    
    def __str__(self):
        s = ""
        min_used_index = min(self.__tape.keys()) 
        max_used_index = max(self.__tape.keys())
        for i in range(min_used_index, max_used_index+1):
            s += self.__tape[i]
        return s    
   
    def __getitem__(self,index):
        if index in self.__tape:
            return self.__tape[index]
        else:
            return self.__blank_symbol

    def __setitem__(self, pos, char):
        self.__tape[pos] = char 

In [ ]:
#
# originally sourced from https://python-course.eu/applications-python/turing-machine.php
# with updates to support serializstion (output) and parsing (input)
#
import string

class TuringMachine(object):
    
  def __init__(self, 
                tape = "", 
                blank_symbol = " ",
                Sigma = " ",
                Gamma = " ",
                states = None,
                initial_state = "",
                final_states = None,
                transition_file = None,
                transition_function = None):
                
    self.__tape = Tape(
       tape_string=tape, 
       blank_symbol=blank_symbol, 
       debug=True)
    
    self.__head_position = 0
    self.__blank_symbol = blank_symbol
    
    self.__Sigma = Sigma
    self.__Gamma = Gamma
    
    self.__states = states
    self.__initial_state =  initial_state
    self.__current_state = initial_state
    
    if (transition_function == None):
        self.read_transition_rules(transition_file)
    else:
        self.__transition_function = transition_function
    
    if final_states == None:
        self.__final_states = set()
    else:
        self.__final_states = set(final_states)


  def get_tape(self): 
    return str(self.__tape)

  def get_tape_blank(self):
     return self.__tape.get_blank_symbol()

  def final(self):
    if self.__current_state in self.__final_states:
        #print('Accepting')
        return True
    else:
        #print('Not Acceping')
        return False
    

  def step(self, debug=False):

    char_under_head = self.__tape[self.__head_position]
    x = (self.__current_state, char_under_head)
    
    if x in self.__transition_function:
        y = self.__transition_function[x]

        if (debug):
          print('transition function: ' + str(x) + ' -> ' + str(y))
          print("tape before operation: " + self.get_tape() + '; [head_pos: ' + str(self.__head_position) + ']')
    
        self.__tape[self.__head_position] = y[1]
        if y[2] == "R":
            self.__head_position += 1
        elif y[2] == "L":
            self.__head_position -= 1
        self.__current_state = y[0]
    else:
       if (debug):
          print('*** WARNING: ' + str(x) + ' not in transition function set ***')
          return False

    if (debug):
      print("tape after operation: " + self.get_tape() + '; [head_pos: ' + str(self.__head_position) + ']')
      print('---------------------------------------------------------')
      
    return True
  
  
  #
  # computes the input_str with a max_ops value as input 
  #
  def compute(self, input_string=None, max_ops=MAX_OPS, debug=False):

    self.__tape = Tape(
       tape_string=input_string, 
       blank_symbol=self.__blank_symbol
       )
    
    if (debug):
       print('TuringMachine::compute(tape=' + self.get_tape() + ', blank_symbol=' + str(self.get_tape_blank()) + ')')
       print('---------------------------------------------------------')

    ops = 0
    while not self.final():
      valid_state = self.step(debug=debug)
      ops += 1

      if not valid_state:
        print('*** not a valid state, therefore not accepting ***')
        return False

      if (ops >= max_ops):
        if (debug):
          print('*** WARNING: not accepting after max_ops('+str(ops)+') operations ***')
        return False
    
    if (debug):
       print("ACCEPTING after " + str(ops) + ' operations.')
       print("TM Tape Output: " + self.get_tape())    

    return True


  #
  # support function for parsing transitions from file
  #
  def split_equation(self, text):
    parts = text.split("=")
    lhs = parts[0].strip()
    rhs = parts[1].strip()
    return lhs, rhs
  

  #
  # support function for parsing transitions from file
  #
  def parse_transition_rule(self, rule):
    
    # Remove whitespace and split into components
    rule = rule.strip().replace(" ", "").replace("δ", "").replace("{", "").replace("}", "")

    lhs, rhs = self.split_equation(rule)    

    lhs = lhs.replace('(', '').replace(')', '')
    rhs = rhs.replace('(', '').replace(')', '')

    # Extract state and symbol from strings
    q, sym = lhs.split(',')
    q_next, sym_next, direction = rhs.split(',')
    
    return (q, sym), (q_next, sym_next, direction)
  
  #
  # read transition rules from file
  # takes filename as input, returns rules
  # 
  def read_transition_rules(self, filename):
    rules = {}
    with open(filename, "r") as f:
        for line in f:
            if line.strip() != "":
                key, value = self.parse_transition_rule(line)
                rules[key] = value
        
        self.__transition_function = rules


  def print(self):
    print("--------------- ** TM Definition ** ---------------")
    print("Q = {" + ", ".join(self.__states) + "}")
    print("Σ = {" + ", ".join(self.__Sigma) + "}")
    print("Γ = {" + ", ".join(self.__Gamma) + "}")
    print("δ : ")
    for key, val in self.__transition_function.items():
        print('   δ'+str(key)+' = {'+str(val)+ str('}'))
    print("q0 = " + self.__initial_state)
    print("b = " + self.__blank_symbol)
    print("F = {" + ", ".join(self.__final_states) + "}")
    print("---------------------------------------------------")


# Examples

## Example 0

In [ ]:
states = {"init", "final"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "init",

accepting_states = ["final"],

transition_function = {("init","0"):("init", "1", "R"),
                       ("init","1"):("init", "0", "R"),
                       ("init","□"):("final","□", "N"),
                       }

final_states = {"final"}

t = TuringMachine(states = states,
                  Gamma = Gamma,
                  Sigma = Sigma,
                  blank_symbol="□",
                  initial_state = "init",
                  final_states = final_states,
                  transition_function=transition_function)

t.print()

if (t.compute("010011001□", debug=DEBUG)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')  
    

## Example 1

In [ ]:
#
# From Module 10 slides:
# For the alphabet (Sigma) = {a, b} design a Turing machine 
# that accepts the language denoted by the regular expression 'a*b*'
#
states = {"q0", "q1"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "q0",
accepting_states = ["q1"], transition_function

final_states = {"q1"}

t = TuringMachine(states=states,
                  Sigma=Sigma,
                  Gamma=Gamma,
                  initial_state="q0",
                  blank_symbol="□",
                  final_states=final_states,
                  transition_function=None,
                  transition_file=FILE_DIR + '/tm10.1')

t.print()

if (t.compute("aaaabbbbbbbb□", debug=DEBUG)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')
    

## Example 2

In [ ]:
#
# From Module 10 slides:
# For the alphabet (Sigma) = {0, 1} design a Turing machine 
# that accepts the language denoted by the regular expression '00*'
#
states = {"q0", "q1"}
Sigma = {"1", "0"}
Gamma = {"1", "0", "□"}

initial_state = "q0",
accepting_states = ["q1"], transition_function

final_states = {"q1"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  initial_state = "q0",
                  final_states = final_states,
                  blank_symbol="□",
                  transition_function=None,
                  transition_file=FILE_DIR + '/tm10.2')

t.print()

if (t.compute("000000000000000□", debug=DEBUG)):
  print("*** Accepting ***")
else:
  print('*** Not Accepting ***')
    

## Example 3

In [ ]:
#
# From Assignment 7a, L = L(aaba*b)
#
states = {"q0", "q1", "q2", "q3", "q4"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□", "x", "y", "z", "w"}

initial_state = "q0",
accepting_states = ["q4"]

final_states = {"q4"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  initial_state = "q0",
                  final_states = final_states,
                  blank_symbol="□",
                  transition_function=None,
                  transition_file=FILE_DIR + '/assign7_1a_04122023.txt')

t.print()

if (t.compute("aabaaaaaaaaaaaaaaab□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

## Example 4

In [ ]:
#
# From Assignment 7a, L = L(aaba*b)
#
states = {"q0", "q1", "q2", "q3", "q4", "q5"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "□"}

initial_state = "q0"
accepting_states = ["q5"]

final_states = {"q5"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=FILE_DIR + '/assign7_1c_04122023.txt')

t.print()

if (t.compute("ababaaaabbba□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')

## Example 5

In [ ]:
#
# From Assignment 7, 1f L = {anbman+m: n >= 0, m >=1 }
# annswer from textbook pg 429 (Section 9.1 question 8f)
#
states = {"q0", "q1", "q2", "q3", "q4", "q5", "q6"}
Sigma = {"a", "b"}
Gamma = {"a", "b", "y", "x", "□"}

initial_state = "q0"
accepting_states = ["q6"]

final_states = {"q6"}

t = TuringMachine(states=states,
                  Gamma=Gamma,
                  Sigma=Sigma,
                  blank_symbol="□",
                  initial_state = initial_state,
                  final_states = final_states,
                  transition_file=FILE_DIR + '/assign7_1f_04122023.txt')

t.print()

if (t.compute("aabbbbaaaaaa□", debug=DEBUG)):
    print("*** Accepting ***")
else:
  print('*** Not Accepting ***')